In [4]:
from RxNorm_API import RxNorm
import pandas as pd
from IPython.display import clear_output
import numpy as np

In [5]:
df = pd.read_csv(r'C:\Users\houstonan\Documents\Projects\RxNorm_Mapping\data\raw\pseudo_medications.csv')
df.head()

,ORDER_MNEM_TXT
0,paracetamol
1,midazolam
2,ondansetron
3,sodium chloride 0.9% intravenous solution 500 ...
4,glyceryl trinitrate


**Preprocessing**

In [6]:
# De-capitalise

df['ORDER_MNEM_TXT'] = df['ORDER_MNEM_TXT'].str.lower()

# Removing unwanted terms
replace = '|'.join(['\d+',
    'ml',
    'mg' ,
    'intravenous',
    'solution',
    '[\(\[].*?[\)\]]',
    'syringe driver -',
    'injection',
    '\.',
    '\%',
    'infusion',
    'topical'])

df['ORDER_MNEM_TXT'] = df['ORDER_MNEM_TXT'].str.replace(replace,'', regex=True)
df = df.drop_duplicates(subset='ORDER_MNEM_TXT', keep="first")
df.head(5)

,ORDER_MNEM_TXT
0,paracetamol
1,midazolam
2,ondansetron
3,sodium chloride
4,glyceryl trinitrate


In [7]:
df.to_csv('Processed_Medications.csv', index=None)

**Mapping**

In [7]:
medications_df = pd.DataFrame(columns = ['input_term','Name','rxcui','SNOMEDCT','MMSL'])
for value in df['ORDER_MNEM_TXT']:
    rxnorm = RxNorm()

    names_ = []
    rxcui_ = []
    snomed_ = []
    mmsl_ = []

    for concept in value.split('+'):

        term = concept
        
        print('Mapping: ' + term)
        
        rxcui = rxnorm.approximate_term(term = term, timeout=5)

        if isinstance(rxcui,list):
            i=0
            while i < len(rxcui):
                snomed, mmsl = rxnorm.get_codes(rxcui = rxcui[i], timeout=5)

                if (snomed == 'NULL') & (mmsl == 'NULL'):
                    selected_rxcui = 'NULL'
                    i+=1
                else:
                    selected_rxcui = rxcui[i]
                    a=i
                    i=len(rxcui)
        else:
            selected_rxcui = rxcui

        if selected_rxcui == 'NULL':
            names_.append('NULL')
            rxcui_.append(selected_rxcui)
            snomed_.append('NULL')
            mmsl_.append('NULL')
        else:
            if a != 0:
                ingredient_rxcui = rxnorm.primary_ingredient(rxcui = selected_rxcui, timeout=5)
                if ingredient_rxcui == 'NULL':
                    names = rxnorm.get_names(rxcui = selected_rxcui, timeout=5)
                    rxcui_.append(selected_rxcui)
                else:
                    names_.append(rxnorm.get_names(rxcui = ingredient_rxcui, timeout=5))
                    rxcui_.append(ingredient_rxcui)
                    snomed, mmsl = rxnorm.get_codes(rxcui = ingredient_rxcui, timeout=5)
                    
            else:
                names_.append(rxnorm.get_names(rxcui = selected_rxcui, timeout=5))
                rxcui_.append(selected_rxcui)
            
            snomed_.append(snomed)
            mmsl_.append(mmsl)


    codes = pd.DataFrame(data = np.array([value, ', '.join(names_), ', '.join(rxcui_),', '.join(snomed_), ', '.join(mmsl_)]).reshape(1,-1), 
        columns = ['input_term','Name','rxcui','SNOMEDCT','MMSL'])
    medications_df = medications_df.append(codes, ignore_index=True)

    clear_output()

In [45]:
print('Missing (%): ', (medications_df["input_term"][medications_df["rxcui"]=='NULL'].count()/medications_df["input_term"].count()).round(3)*100)

Missing (%):  12.7


In [46]:
medications_df.to_csv('Medication_Post_Mapping.csv', index = False)

## Comparison with SNAPPER

In [103]:
import pandas as pd
import math

In [104]:
RxNorm_df = pd.read_csv(r'C:\Users\houstonan\Documents\Projects\RxNorm_Mapping\data\processed\RxNorm_Mapping.csv')
SNAPPER_df = pd.read_csv(r'C:\Users\houstonan\Documents\Projects\RxNorm_Mapping\data\processed\Snapper_Mapping.csv')

Check if SNAPPER SNOMED code is contained within RxNorm SNOMED codes

In [124]:
comparable_mapping = []
for i in range(RxNorm_df.shape[0]):
    if isinstance(RxNorm_df.loc[i]['SNOMEDCT'], str):
        if math.isnan(SNAPPER_df['Snapper_Snomed'][i]) == False:
            if str(int(round(SNAPPER_df['Snapper_Snomed'])[i])) in RxNorm_df.loc[i]['SNOMEDCT']:
                comparable_mapping.append(1)
            else:
                comparable_mapping.append(0)
        else:
            comparable_mapping.append(-1)
    else: 
        if math.isnan(SNAPPER_df['Snapper_Snomed'][i]) == False:
            comparable_mapping.append(-2)
        else:
            comparable_mapping.append(-3)

print("Comparable Mappings (%): ", np.round((comparable_mapping.count(1)/(comparable_mapping.count(1)+comparable_mapping.count(0)))*100))

Comparable Mappings (%):  88.0


If mapping is the same take the RxNorm codes, if not, take the SNOMED codes. If one mapping system can't find a concept, take the one that can. If neither can, return NULL.

In [ ]:
merged_mappings = pd.DataFrame(columns=['input_term', 'retrieved_term', 'SNOMED_Code'])

for i in range(len(comparable_mapping)):
    input_term = RxNorm_df['input_term'].loc[i]
    if comparable_mapping[i] == 1 or comparable_mapping[i] == -1:
        retrieved_term = RxNorm_df['Name'].loc[i]
        SNOMED_Code = RxNorm_df['SNOMEDCT'].loc[i]

    if comparable_mapping[i] == 0 or comparable_mapping[i] == -2:
        retrieved_term = SNAPPER_df['Snapper_Mapping'].loc[i]
        SNOMED_Code = str(int(round(SNAPPER_df['Snapper_Snomed'].loc[i])))

    if comparable_mapping[i] == -3:
        retrieved_term = 'NULL'
        SNOMED_Code = 'NULL'

    merged_mappings = merged_mappings.append(pd.DataFrame(data=np.array([input_term,retrieved_term,SNOMED_Code]).reshape(1,-1), columns = ['input_term', 'retrieved_term', 'SNOMED_Code']))
merged_mappings = merged_mappings.reset_index(drop=True)

In [157]:
print('Remaining Missing (%): ', round(((merged_mappings['SNOMED_Code'] == 'NULL').sum()/merged_mappings.shape[0])*100,2))

Remaining Missing (%):  2.66


In [161]:
merged_mappings.head(60)

,input_term,retrieved_term,SNOMED_Code
0,paracetamol,acetaminophen,"387517004, 90332006"
1,midazolam,midazolam,"26800000, 373476007"
2,ondansetron,ondansetron,"108418007, 372487007"
3,sodium chloride,sodium chloride,"387390002, 70379000, 786818003"
4,glyceryl trinitrate,nitroglycerin,"387404004, 71759000"
5,heparin,heparin,"372877000, 84812008"
6,isosorbide dinitrate,isosorbide dinitrate,387332007
7,metaraminol,metaraminol,"372728001, 41015006"
8,morphine,morphine,"373529000, 73572009"
9,compound sodium lactate,sodium lactate,"387306001, 55015008"


Input terms neither mapping system can find

In [148]:
for i in range(merged_mappings.shape[0]):
    if merged_mappings['retrieved_term'].loc[i] == 'NULL':
        print(merged_mappings['input_term'].loc[i])

freetext medication
compliance aid
ocular lubricant
mechanical prophylaxis
discharge medication approved
bicarbonate dialysate 
prothrombin complex
freetext medication 
fresubin  kcal bottle
benzalkonium/dimeticone/hc/nystatin top
energy dense snack
energy dense snacks
larvae therapy
ed snack
ranibizumab ophthalmic
aerochamber
nourishing snack
flaminal hydro
insulin pump therapy placeholder
bedtime snacks
dextroenergy
volumatic
trace elements
dexamethasone/framycetin/gramicid ophth
